In [5]:
with open("/Users/zhengxiao/Downloads/generation-large-cnndm/test.jsonl.hyps") as file:
    hyps = file.readlines()
    hyps = [line.strip() for line in hyps]

with open("/Users/zhengxiao/Downloads/generation-large-cnndm/test.jsonl.refs") as file:
    refs = file.readlines()
    refs = [line.strip() for line in refs]

In [8]:
import string

def _is_digit(w):
    for ch in w:
        if not (ch.isdigit() or ch == ','):
            return False
    return True


def fix_tokenization(text):
    input_tokens = text.split()
    output_tokens = []
    has_left_quote = False
    has_left_single_quote = False

    i = 0
    prev_dash = False
    while i < len(input_tokens):
        tok = input_tokens[i]
        flag_prev_dash = False
        if tok == "\"":
            if has_left_quote:
                output_tokens.append("''")
            else:
                output_tokens.append("``")
            has_left_quote = not has_left_quote
            i += 1
        elif tok == "'" and len(output_tokens) > 0 and output_tokens[-1].endswith("n") and i < len(input_tokens) - 1 and \
                input_tokens[i + 1] == "t":
            output_tokens[-1] = output_tokens[-1][:-1]
            output_tokens.append("n't")
            i += 2
        elif tok == "'" and i < len(input_tokens) - 1 and input_tokens[i + 1] in ("s", "d", "ll"):
            output_tokens.append("'" + input_tokens[i + 1])
            i += 2
        elif tok == "'":
            if has_left_single_quote:
                output_tokens.append("'")
            else:
                output_tokens.append("`")
            has_left_single_quote = not has_left_single_quote
            i += 1
        elif tok == "." and i < len(input_tokens) - 2 and input_tokens[i + 1] == "." and input_tokens[i + 2] == ".":
            output_tokens.append("...")
            i += 3
        elif tok == "," and len(output_tokens) > 0 and _is_digit(output_tokens[-1]) and i < len(
                input_tokens) - 1 and _is_digit(input_tokens[i + 1]):
            # $ 3 , 000 -> $ 3,000
            output_tokens[-1] += ',' + input_tokens[i + 1]
            i += 2
        elif tok == "." and len(output_tokens) > 0 and output_tokens[-1].isdigit() and i < len(input_tokens) - 1 and \
                input_tokens[i + 1].isdigit():
            # 3 . 03 -> $ 3.03
            output_tokens[-1] += '.' + input_tokens[i + 1]
            i += 2
        elif tok == "." and len(output_tokens) > 0 and len(output_tokens[-1]) == 1 and output_tokens[
            -1].isupper() and i < len(input_tokens) - 2 and len(input_tokens[i + 1]) == 1 and input_tokens[
            i + 1].isupper() and input_tokens[i + 2] == '.':
            # U . N . -> U.N.
            k = i + 3
            while k + 2 < len(input_tokens):
                if len(input_tokens[k + 1]) == 1 and input_tokens[k + 1].isupper() and input_tokens[k + 2] == '.':
                    k += 2
                else:
                    break
            output_tokens[-1] += ''.join(input_tokens[i:k])
            i += 2
        elif tok == "-":
            if i < len(input_tokens) - 1 and input_tokens[i + 1] == "-":
                output_tokens.append("--")
                i += 2
            elif i == len(input_tokens) - 1 or i == 0:
                output_tokens.append("-")
                i += 1
            elif output_tokens[-1] not in string.punctuation and input_tokens[i + 1][0] not in string.punctuation:
                output_tokens[-1] += "-"
                i += 1
                flag_prev_dash = True
            else:
                output_tokens.append("-")
                i += 1
        elif prev_dash and len(output_tokens) > 0 and tok[0] not in string.punctuation:
            output_tokens[-1] += tok
            i += 1
        else:
            output_tokens.append(tok)
            i += 1
        prev_dash = flag_prev_dash
    return " ".join(output_tokens)


def count_tokens(tokens):
    counter = {}
    for t in tokens:
        if t in counter.keys():
            counter[t] += 1
        else:
            counter[t] = 1
    return counter


def get_f1(text_a, text_b):
    tokens_a = text_a.lower().split()
    tokens_b = text_b.lower().split()
    if len(tokens_a) == 0 or len(tokens_b) == 0:
        return 1 if len(tokens_a) == len(tokens_b) else 0
    set_a = count_tokens(tokens_a)
    set_b = count_tokens(tokens_b)
    match = 0
    for token in set_a.keys():
        if token in set_b.keys():
            match += min(set_a[token], set_b[token])
    p = match / len(tokens_a)
    r = match / len(tokens_b)
    return 2.0 * p * r / (p + r + 1e-5)


def remove_duplicate(l_list, duplicate_rate):
    tk_list = [l.lower().split() for l in l_list]
    r_list = []
    history_set = set()
    for i, w_list in enumerate(tk_list):
        w_set = set(w_list)
        if len(w_set & history_set) / len(w_set) <= duplicate_rate:
            r_list.append(l_list[i])
        history_set |= w_set
    return r_list


In [12]:
from rouge_score import rouge_scorer
metric = "rouge-1"
duplicate_rate = 0.7
metric_dict = {"rouge-1": "rouge1", "rouge-2": "rouge2", "rouge-l": "rougeLsum"}
ref_list = []
for ref in refs:
    ref = ref.strip().split('[SEP]')
    ref = [fix_tokenization(sentence) for sentence in ref]
    ref = "\n".join(ref)
    ref_list.append(ref)
pred_list = []
for prediction in hyps:
    buf = []
    for sentence in prediction.strip().split("[SEP]"):
        sentence = fix_tokenization(sentence)
        if any(get_f1(sentence, s) > 1.0 for s in buf):
            continue
        s_len = len(sentence.split())
        if s_len <= 4:
            continue
        buf.append(sentence)
    if duplicate_rate and duplicate_rate < 1:
        buf = remove_duplicate(buf, duplicate_rate)
    line = "\n".join(buf)
    pred_list.append(line)
scorer = rouge_scorer.RougeScorer([metric_dict[metric]], use_stemmer=True)
scores = [scorer.score(pred, ref) for pred, ref in zip(pred_list, ref_list)]
scores = [score[metric_dict[metric]].fmeasure for score in scores]
scores = sum(scores) / len(scores)
print(scores)

0.42271185970285236


In [20]:
import random
idx = random.randrange(len(ref_list))
print(ref_list[idx])
print("")
print(pred_list[idx])

the raiders stripped off before entering the esso station in caterham .
police have released cctv images of two of the three would-be raiders .
the men escaped with one staff member 's mobile phone but no cash .
a third man has been arrested and is on police bail following the incident .

police in surrey have released cctv of two raiders who stripped-off their shirts before attempting to rob an esso petrol station .
the men , who were both in their 20s , forced their way into the manager 's office of the filling station in town end , caterham , surrey , looking for cash .
they were disturbed by a member of the garage staff and fled .
a third man , aged in his 20s , had been arrested and released on police bail while the investigation continues .


In [1]:
from data_utils.tokenization import make_tokenizer
tokenizer = make_tokenizer("BertWordPieceTokenizer", None, None, None, "bert-large-uncased")

In [4]:
tokenizer.EncodeAsIds("[MASK]").tokenization

[103]

In [20]:
fix_tokenization(hyps[5])

"the student was identified during an investigation by campus police and the office of student affairs . officials are still trying to determine if other people were involved . students march and chant , `` we are not afraid . we stand together , '' after pictures of the noose are passed around on social media ."

In [63]:
refs = [tokenizer.DecodeIds(tokenizer.EncodeAsIds(ref).tokenization) for ref in refs]

In [21]:
hyps = [fix_tokenization(hyp) for hyp in hyps]
refs = [fix_tokenization(ref) for ref in refs]

In [6]:
hyps = [hyp.replace("< unk >", "unk") for hyp in hyps]
refs = [ref.replace("< unk >", "unk") for ref in refs]

In [7]:
with open("./generation-large-ggw_test/test.jsonl.hyps_new", "w") as output:
    for hyp in hyps:
        output.write(hyp + "\n")
with open("./generation-large-ggw_test/test.jsonl.refs_new", "w") as output:
    for ref in refs:
        output.write(ref + "\n")

In [56]:
def add_line_break(text):
    text = text.split(" . ")
    text = " . \n ".join(text)
    return text

In [58]:
hyps = [add_line_break(hyp) for hyp in hyps]
refs = [add_line_break(ref) for ref in refs]

In [59]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(hyps, refs, avg=True)
print(scores)

{'rouge-1': {'f': 0.41868113530622203, 'p': 0.39858987750197444, 'r': 0.46918404550355053}, 'rouge-2': {'f': 0.1992393909684264, 'p': 0.19046146701483258, 'r': 0.22262230082962056}, 'rouge-l': {'f': 0.4024112886450365, 'p': 0.3852430050046152, 'r': 0.44030556796284664}}


In [49]:
import random
rand_id = random.randrange(len(hyps))
print(hyps[rand_id])
print(refs[rand_id])

tennessee judge lila statom has become a viral sensation thanks to her no - nonsense, brutal take down of a gang member who dared try to claim an area as'his hood'after o'shae smith told the judge he shot a rival gang member for being'in his hood,'she had no problem telling him the area didn't actually belong to him. smith has been charged with attempted first - degree murder for shooting kendre allen, 18, in the hip in late march.
o'shae smith told judge lila statom that he shot a rival gang member for being'in his hood'last month. statom told him east court lakes, the housing project where the shooting took place, belonged only to the hard - working people who live there. the chattanooga area has been ravaged by gang violence. statom said she wanted to show smith that the courtroom was an area he couldn't intimidate or control.
